In [1]:
from imutils import face_utils
import numpy as np
import imutils
import cv2
import dlib
import collections
import matplotlib.pyplot as plt
import pandas as pd
from os import path
%matplotlib inline  

In [2]:
shape_predictor = "shape_predictor_68_face_landmarks.dat"

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

In [42]:
def writeImage(part, imagePath, imageName, partName):
    image = imagePath + imageName + ".jpg"
    image = cv2.imread(image)
#     image = imutils.resize(image, width=500)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    (x, y, w, h) = cv2.boundingRect(part)
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if w < 0:
        w = 0
    if h < 0:
        h = 0
    roi = image[y:y + h, x:x + w]
#     roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
#     cv2.imwrite(imagePath + imageName + "_" + partName + "_colored.jpg", roi)
    cv2.imwrite(imagePath + imageName + "_" + partName + ".jpg", roi)

In [38]:
def segmentLandmarks(imagePath, imageName):  
    # load the input image, resize it, and convert it to grayscale
    image = imagePath + imageName + ".jpg"
    image = cv2.imread(image)
#     image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)

    faceparts = dict()
    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # loop over the face parts individually
        for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            faceparts[name] = shape[i:j]
            # clone the original image so we can draw on it, then
            # display the name of the face part on the image
#             clone = image.copy()
#             cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
#                 0.7, (0, 0, 255), 2)

#             # loop over the subset of facial landmarks, drawing the
#             # specific face part
#             for (x, y) in shape[i:j]:
#                 cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
#             # extract the ROI of the face region as a separate image
#             (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
#             if x < 0:
#                 x = 0
#             if y < 0:
#                 y = 0
#             if w < 0:
#                 w = 0
#             if h < 0:
#                 h = 0
#             roi = image[y:y + h, x:x + w]
# #             print(shape[i:j])
#             roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)

# #             print landmarks
#             cv2.imshow("ROI", roi)
#             cv2.imshow("Image", clone)
#             cv2.waitKey(0)

#   store the landmarks
    if faceparts == {}:
        return False
#     rightEyeParts = np.concatenate((faceparts['right_eyebrow'], faceparts['right_eye']))
#     writeImage(rightEyeParts, imagePath, imageName, 'right_eye')
#     leftEyeParts = np.concatenate((faceparts['left_eyebrow'], faceparts['left_eye']))
#     writeImage(leftEyeParts, imagePath, imageName, 'left_eye')
#     mouth_nose = np.concatenate((faceparts['nose'], faceparts['mouth']))
#     writeImage(mouth_nose, imagePath, imageName, 'mouth')
    writeImage(np.array(shape), imagePath, imageName, 'face')
    return True

In [36]:
segmentLandmarks('./database/SAMM/Cropped/006/006_5_9/', '006_5258')

l


True

In [39]:
def loadAllFile(database, databaseName, i, j):
    fileName = ''
    imageName = ''
    if databaseName == 'CASME2':
        fileName = 'sub0' + str(database.Subject[i])
        if database.Subject[i] >= 10:
            fileName = 'sub' + str(database.Subject[i])
        fileName = fileName + '/' + database.Filename[i] + '/'
        imageName = 'reg_img'
    if databaseName == 'CASME':
        fileName = 'sub0' + str(database.Subject[i])
        if database.Subject[i] >= 10:
            fileName = 'sub' + str(database.Subject[i])
        fileName = fileName + '/' + database.Filename[i] + '/'
        imageName = 'reg_' + database.Filename[i] + '-' 
        if not(path.exists('./database/' + databaseName + '/Cropped/' + fileName + imageName + str(int(database.OnsetFrame[i]) + j) + '.jpg')):
            foreName = '-0' 
            if int(database.OnsetFrame[i]) + j < 10:
                foreName = '-00' 
            imageName = 'reg_' + database.Filename[i] + foreName
    if databaseName == 'SAMM':
        foreName = '00' + str(database.Subject[i])
        if database.Subject[i] >= 10:
            foreName = '0' + str(database.Subject[i])
        fileName = foreName + '/' + database.Filename[i] + '/'
        imageName = foreName + '_' 
#                 print('./database/' + databaseName + '/Cropped/' + fileName + imageName + '.jpg')
        if not(path.exists('./database/' + databaseName + '/Cropped/' + fileName + imageName + str(int(database.OnsetFrame[i]) + j) + '.jpg')):
            imageName = '0' 
            if int(database.OnsetFrame[i]) + j < 1000:
                imageName = '00' 
            imageName = foreName + '_' + imageName
        if not(path.exists('./database/' + databaseName + '/Cropped/' + fileName + imageName + str(int(database.OnsetFrame[i]) + j) + '.jpg')):
            imageName = '0' 
            if int(database.OnsetFrame[i]) + j < 100:
                imageName = '00'
            imageName = foreName + '_' + imageName
    imagePath = './database/' + databaseName + '/Cropped/' + fileName
    return imagePath, imageName

In [40]:
def segmentAll(database, databaseName, emotions):
    path = []
    image = []
    onset = []
    apex = []
    current = []
    AU = []
    Emotions = []
    for i in range(len(database)):
        emotion = database['Emotion'][i]
        if not(emotion in emotions):
            continue
        if database.ApexFrame[i] == "/":
            continue
        for j  in range(int(database.ApexFrame[i]) - int(database.OnsetFrame[i]) + 1):
            imagePath, imageName = loadAllFile(database, databaseName, i, j)
#             print(imagePath, imageName)
            if not(segmentLandmarks(imagePath, imageName + str(int(database.OnsetFrame[i]) + j))):
                continue
            path = path + [imagePath]
            image = image + [imageName]
            onset = onset + [database.OnsetFrame[i]]
            apex = apex + [database.ApexFrame[i]]
            current = current + [str(int(database.OnsetFrame[i]) + j)]
            AU = AU + [database.AU[i]]
            Emotions = Emotions + [emotion]
        print('finish ' + imagePath + imageName + str(int(database.OnsetFrame[i]) + j))
    d = {
        'path': path,
        'image': image,
        'onset': onset,
        'apex': apex,
        'current': current,
        'AU': AU,
        'emotion': Emotions
    }
    df = pd.DataFrame(data = d)
    return df

In [43]:
casme2 = pd.read_csv('./database/CASME2/casme2.csv')
df = segmentAll(casme2, 'CASME2', ['happiness', 'disgust', 'surprise', 'fear'])
casme = pd.read_csv('./database/CASME/casme.csv')
df2 = segmentAll(casme, 'CASME', ['happiness', 'disgust', 'surprise', 'fear'])
samm = pd.read_csv('./database/SAMM/samm.csv')
df3 = segmentAll(samm, 'SAMM', ['Happiness', 'Disgust', 'Surprise', 'Fear'])

finish ./database/CASME2/Cropped/sub01/EP02_01f/reg_img59
finish ./database/CASME2/Cropped/sub01/EP19_05f/reg_img416
finish ./database/CASME2/Cropped/sub01/EP19_06f/reg_img71
finish ./database/CASME2/Cropped/sub02/EP09_01/reg_img56
finish ./database/CASME2/Cropped/sub02/EP11_01/reg_img69
finish ./database/CASME2/Cropped/sub02/EP13_04/reg_img51
finish ./database/CASME2/Cropped/sub02/EP14_01/reg_img114
finish ./database/CASME2/Cropped/sub03/EP01_2/reg_img69
finish ./database/CASME2/Cropped/sub03/EP07_04/reg_img81
finish ./database/CASME2/Cropped/sub03/EP09_03/reg_img76
finish ./database/CASME2/Cropped/sub03/EP18_06/reg_img81
finish ./database/CASME2/Cropped/sub03/EP19_08/reg_img121
finish ./database/CASME2/Cropped/sub04/EP12_02f/reg_img387
finish ./database/CASME2/Cropped/sub04/EP13_06f/reg_img108
finish ./database/CASME2/Cropped/sub05/EP02_07/reg_img66
finish ./database/CASME2/Cropped/sub05/EP03_01/reg_img81
finish ./database/CASME2/Cropped/sub05/EP03_06/reg_img66
finish ./database/CASM

finish ./database/CASME/Cropped/sub08/EP12_2_2/reg_EP12_2_2-047
finish ./database/CASME/Cropped/sub08/EP12_2_3/reg_EP12_2_3-023
finish ./database/CASME/Cropped/sub08/EP12_2_5/reg_EP12_2_5-016
finish ./database/CASME/Cropped/sub08/EP12_2_7/reg_EP12_2_7-040
finish ./database/CASME/Cropped/sub08/EP12_2_9/reg_EP12_2_9-066
finish ./database/CASME/Cropped/sub08/EP12_4_1/reg_EP12_4_1-023
finish ./database/CASME/Cropped/sub08/EP12_4_10/reg_EP12_4_10-026
finish ./database/CASME/Cropped/sub08/EP12_4_2/reg_EP12_4_2-011
finish ./database/CASME/Cropped/sub08/EP12_4_3/reg_EP12_4_3-014
finish ./database/CASME/Cropped/sub08/EP12_4_4/reg_EP12_4_4-009
finish ./database/CASME/Cropped/sub08/EP12_4_5/reg_EP12_4_5-014
finish ./database/CASME/Cropped/sub08/EP12_4_6/reg_EP12_4_6-009
finish ./database/CASME/Cropped/sub08/EP12_4_7/reg_EP12_4_7-021
finish ./database/CASME/Cropped/sub08/EP12_4_8/reg_EP12_4_8-012
finish ./database/CASME/Cropped/sub08/EP12_4_9/reg_EP12_4_9-019
finish ./database/CASME/Cropped/sub08/

In [10]:
newDf = df.append(df2, ignore_index=True).append(df3, ignore_index = True)

In [11]:
newDf.to_csv('./database/segmented_parts_colored.csv')